In [2]:
import pandas as pd
import yfinance as yf
from datetime import date, timedelta
from matplotlib import pyplot as plt

C:\Users\ashkm\AppData\Local\Temp\ipykernel_48680\2445797640.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [8]:
import numpy as np

In [3]:
def corr_finder(a, b, t):
    df = yf.download([a,b], start=date.today() - timedelta(5000), end=date.today() + timedelta(1), interval='1d')
    df = df['Adj Close'].resample('D').last().dropna()
    correlation = df[a].pct_change().corr(df[b].pct_change())
    return correlation

variables = ['^GSPC', '^TNX', 'USO', 'JPY=X']
correlations = {}

for a in variables:
    for b in [x for x in variables if x != a]:
        correlations[f'{a}:{b}'] = corr_finder(a, b, 100)

df = pd.DataFrame.from_dict(correlations, orient='index', columns=['correlation'])
df.sort_values('correlation', ascending=False)

C:\Users\ashkm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\ashkm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  2 of 2 completed
C:\Users\ashkm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in 

,correlation
USO:^GSPC,0.352515
^GSPC:USO,0.352515
^GSPC:^TNX,0.337368
^TNX:^GSPC,0.337368
^TNX:USO,0.226596
USO:^TNX,0.226596
JPY=X:^TNX,0.029973
^TNX:JPY=X,0.029972
JPY=X:^GSPC,-0.017991
^GSPC:JPY=X,-0.017991


In [59]:
ticker = yf.Ticker('JPY=X')
df = ticker.history(period = '6mo', interval='1d')

C:\Users\ashkm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [60]:
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-08-23 00:00:00+01:00,145.763000,145.813004,144.580002,145.763000,0,0.0,0.0
2023-08-24 00:00:00+01:00,144.673004,145.947006,144.621002,144.673004,0,0.0,0.0
2023-08-25 00:00:00+01:00,146.067001,146.604996,145.733994,146.067001,0,0.0,0.0
2023-08-28 00:00:00+01:00,146.531006,146.716003,146.278000,146.531006,0,0.0,0.0
2023-08-29 00:00:00+01:00,146.464005,147.343994,145.852005,146.464005,0,0.0,0.0


In [115]:
def get_stock_data_to_df(ticker):
    ticker = yf.Ticker(str(ticker))
    df = ticker.history(period = '6mo', interval='1d')
    df.reset_index(inplace=True)
    df["timestamp_processed"] = [e.strftime('%Y-%m-%d') for e in df["Date"]]
    return df

In [108]:
def stationaryize(df : pd.DataFrame, time) -> pd.DataFrame:
    df['stationary'] = (np.log(df[time])).diff()
    df = df.dropna(how = 'any', axis = 0, )
    return df

In [109]:
def get_clean_data(ticker, time) -> pd.DataFrame:
    return stationaryize(get_stock_data_to_df(ticker.upper()), time)

In [116]:
aapl = get_clean_data('aapl', "Close")

C:\Users\ashkm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


TypeError: strftime() takes exactly 2 positional arguments (1 given)

In [104]:
columns = aapl.columns.tolist()
print(columns)

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'stationary']


In [105]:
def merge_df(ticker1, ticker2, open_or_close):
    df1 = get_clean_data(ticker1, open_or_close)
    # print("df of  " + ticker1 + " " + str(np.shape(df1)))
    print("df of " + ticker1)
    print(df1.head())
    df2 = get_clean_data(ticker2, open_or_close)
    # print("df of  " + ticker2 + " " + str(np.shape(df2)))
    print("df of " + ticker2)
    print(df2.head())
    # return pd.merge(df1[['stationary']], df2[['stationary']], left_index=True, right_index=True, how='right').rename(columns={'stationary_x': ticker1, 'stationary_y': ticker2}).dropna(how='any')
    return pd.merge(df1[['timestamp_processed', 'stationary']], df2[['timestamp_processed', 'stationary']], on='timestamp_processed', how='right').rename(columns = {'stationary_x': ticker1, 'stationary_y': ticker2}).dropna(how='any')


In [106]:
merge_df('^TNX', 'JPY=X', 'Close')

df of ^TNX
                       Date   Open   High    Low  Close  Volume  Dividends  \
1 2023-08-24 00:00:00-04:00  4.223  4.239  4.190  4.235       0        0.0   
2 2023-08-25 00:00:00-04:00  4.235  4.285  4.204  4.239       0        0.0   
3 2023-08-28 00:00:00-04:00  4.232  4.234  4.194  4.212       0        0.0   
4 2023-08-29 00:00:00-04:00  4.220  4.243  4.106  4.122       0        0.0   
5 2023-08-30 00:00:00-04:00  4.137  4.155  4.087  4.118       0        0.0   

   Stock Splits  stationary  
1           0.0    0.008775  
2           0.0    0.000944  
3           0.0   -0.006390  
4           0.0   -0.021599  
5           0.0   -0.000971  
df of JPY=X
                       Date        Open        High         Low       Close  \
1 2023-08-24 00:00:00+01:00  144.673004  145.947006  144.621002  144.673004   
2 2023-08-25 00:00:00+01:00  146.067001  146.604996  145.733994  146.067001   
3 2023-08-28 00:00:00+01:00  146.531006  146.716003  146.278000  146.531006   
4 2023-08-29

C:\Users\ashkm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\ashkm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


,^TNX,JPY=X
1,0.008775,-0.007506
2,0.000944,0.009589
3,-0.006390,0.003172
4,-0.021599,-0.000457
5,-0.000971,-0.003255
...,...,...
128,NaN,0.000967
129,NaN,0.001192
130,NaN,-0.002205
131,NaN,0.002771


In [90]:
from statsmodels.tsa.stattools import grangercausalitytests

In [91]:
def granger_causality(ticker1, ticker2, time, lag):
    df = merge_df(ticker1, ticker2, time)
    # if len(df[ticker1]) != len(df[ticker2]):
    minrows = min(len(df[ticker1]), len(df[ticker2]))
    # df = df.iloc[:,:minrows]
    print(np.shape(df))
    return grangercausalitytests(df[[ticker1, ticker2]], lag)


In [92]:
# granger_causality('AAPL', 'TSLA', 'Close', 4)

In [93]:
granger_causality('^TNX', 'JPY=X', 'Close', 4)

df of  ^TNX (126, 8)
df of ^TNX
                            Open   High    Low  Close  Volume  Dividends  \
Date                                                                       
2023-08-24 00:00:00-04:00  4.223  4.239  4.190  4.235       0        0.0   
2023-08-25 00:00:00-04:00  4.235  4.285  4.204  4.239       0        0.0   
2023-08-28 00:00:00-04:00  4.232  4.234  4.194  4.212       0        0.0   
2023-08-29 00:00:00-04:00  4.220  4.243  4.106  4.122       0        0.0   
2023-08-30 00:00:00-04:00  4.137  4.155  4.087  4.118       0        0.0   

                           Stock Splits  stationary  
Date                                                 
2023-08-24 00:00:00-04:00           0.0    0.008775  
2023-08-25 00:00:00-04:00           0.0    0.000944  
2023-08-28 00:00:00-04:00           0.0   -0.006390  
2023-08-29 00:00:00-04:00           0.0   -0.021599  
2023-08-30 00:00:00-04:00           0.0   -0.000971  
df of  JPY=X (132, 8)
df of JPY=X
                       

C:\Users\ashkm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\ashkm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


ValueError: x contains NaN or inf values.